In [ ]:
from datetime import datetime
import sys; sys.path.append('..')

import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from lib.data.santa_fe import load_santa_fe
from lib.utils.time_series import create_time_series_inputs_targets

# 3. Time-Series Prediction with a Long Short-Term Memory Network

Long Short Term Memory networks, usually just called "LSTMs", are a special kind of RNN, capable of learning long-term dependencies ([Hochreiter and Schmidhuber, 1997](https://www.nichenjie.com/mdres/posts/2018/lstm/2604.pdf)). Wheras the MLP-based recurrent model we discussed in the previous notebook takes in an entire (fixed-size) time series as input, an LSTM takes in the time series data points *one at a time*. For each data point, the LSTM cell produces two outputs: a hidden state $h_t$ and a cell state $c_t$. The hidden state and the cell state correspond to short-term and long-term memory, respectively. Apart from the time-series data point $x_t$, an LSTM cell takes two other inputs: the previous hidden state $h_{t-1}$ and the previous cell state $c_{t-1}$.

The hidden state $h_t$ produced by the LSTM can be interpreted as a vector-representation of the predicted data point at the next time instant. In our implementation below, we will project $h_t$ onto a single scalar, which will then be the predicted data point itself.

As you can see in the figure below, the LSTM cell modifies the previous cell state and hidden state. This happens via a *gating* mechanism. Based on the previous hidden state $h_{t-1}$ and the current data point $x_t$, the LSTM cell computes three *gating vectors* (the outputs of each $\sigma$ operation) that contain elements between 0 and 1. When element-wise multiplying such a gating vector with another vector, each element of the gating vector acts as a little *gate* that allows data of the other vector to (partially) pass through or not. A zero blocks the data, a one lets the data through.

For example, the *cell gate* is the gate produced by the $\sigma$ operation at the left-hand side of the figure. It chooses which elements of the previous cell state $c_{t-1}$ will be kept and which will be removed. The *input gate* (second from the left) chooses which information will be added to the new cell state, while the *output gate* (right-hand side $\sigma$) chooses which information of the new cell state $c_t$ (after applying $\tanh$) to use for $h_t$.

<img src="https://upload.wikimedia.org/wikipedia/commons/9/93/LSTM_Cell.svg" style="max-width: 500px;"/>

Note that **the same cell is used at each time instant**, but always with **new inputs** $h_{t-1}$, $c_{t-1}$ and $x_t$.

The output of every gate is produced by passing $x_t$ and $h_{t-1}$ each through their own fully-connected layer, adding the results and applying a sigmoid operation. For producing the result of the $\tanh$ branch in the middle of the figure, $x_t$ and $h_{t-1}$ are also each passed through their own fully-connected layer. The results are again added together but passed through a $\tanh$ operation instead of a sigmoid.

As such, the LSTM cell contains **8 fully-connected layers** with learnable parameters. These parameters can be trained via a gradient descent-based optimization method.

## 3.1 LSTM-based Model for Time-Series Prediction

We have implemented an LSTM-based regression model below. The model employs [`nn.LSTM`](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) to process a sequence of inputs. As you can see in the `forward()` method, the resulting hidden state vector is passed through a fully-connected layer, which transforms it into a prediction for the next time instant.

In [ ]:
class LSTMRegressor(nn.Module):
    def __init__(self, hidden_size, input_size=1, num_layers=1):
        """
        Args:
            hidden_size (int): The number of features to use for the hidden state.
            input_size (int): The number of dimensions a measurement has at a single time step.
            num_layers (int): Number of recurrent layers. E.g., setting `num_layers=2` would
                mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM
                taking in the hidden states of the first LSTM and computing the final results.
        """
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Initialize the LSTM
        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
        )

        # Initialize the fully-connected layer
        self.linear = nn.Linear(
            in_features=self.hidden_size,
            out_features=self.input_size
        )

    def forward(self, x):
        """
        Args:
            x (torch.tensor): tensor of shape `(N, L, H_in)`, `(N, L)` or `(L,)` with `N`
                the batch size, `L` the sequence length and `H_in` the input size. If
                the dimensions of `L` and/or `H_in` are not given, they are assumend to
                be 1.
        """
        orig_ndim = x.ndim

        if orig_ndim == 1:
            # Assume batch size and input size are 1
            x = x[None, ..., None]
        if orig_ndim == 2:
            # Assume input size is 1
            x = x[..., None]

        N, L, H_in = x.shape

        assert self.input_size == H_in

        # Reshape to (L, N, H_in) for LSTM input
        x = x.permute(1, 0, 2)

        # `output` contains hidden state from last layer for each sequence element
        # `h_n` contains the hidden state from all layers for the last sequence element
        # `c_n` contains the cell state from all layers for the last sequence element
        output, (h_n, c_n) = self.lstm(x)

        # Select the last hidden state from the last element...
        h_last = h_n[-1]  # (Note: same as output[-1])

        # ...and pass it through a fully-connected layer
        out = self.linear(h_last)

        if orig_ndim == 1:
            # Remove batch dimension
            out = out[0, :]

        return out

The number of features in the hidden state can be controlled with the `hidden_size` argument of the constructor. For example, one can create an LSTM-based model with a hidden state vector of length 128 as follows:

```python
# Create the LSTM model
lstm = LSTMRegressor(hidden_size=128)
```

Once you have created the model, you can use it in a similar way as the MLP from the [previous notebook](./2_nn_time_series.ipynb):

```python
# Predict the next time series value
y_next = lstm(time_series)
```

Where `time_series` is a tensor of shape $(N, L)$ with $N$ the batch size and $L$ the length of a single time series sequence. Then, `y_next` will be a tensor of shape $(N, 1)$ containing the next predicted value for each sequence. Just like with the MLP, we can compare these predictions with the ground-truth values and compute (and backpropagate) the loss.

## 3.2 Making Time-Series Predictions

In the previous notebook, you have implemented the function `run_recurrent_model()` to produce time-series predictions for an MLP-based recurrent model. One limitation of such a model is that the sequence length is *fixed*, i.e., it is equal to the number of input neurons of the MLP. This is an important difference with an LSTM-based model. An LSTM takes in a data point at a single time instant, the previous hidden state and the previous cell state and produces the next hidden state and the next cell state. There is no restriction on the length of the time-series sequence.

Paste your `run_recurrent_model()` implementation of the previous notebook below and modify it so that the length of the time-series grows as the model predicts more data points (instead of dropping the oldest data point).

In [ ]:
## Write your modified run_recurrent_model() implementation here ##

## 3.3 Exercise

Reuse your training code from the [previous notebook](./2_nn_time_series.ipynb) to model the Santa Fe dataset with the `LSTMRegressor` class.

- Train the LSTM-based model and explain the design process. Discuss how the model looks, the parameters that you tune,... What is the effect of changing the lag value for the LSTM network?
- Afterwards, try to predict the validation set. Use your newly implemented `run_recurrent_model()` function for this.
- Compare results of the recurrent neural network of the previous notebook with the LSTM. Which model do you prefer and why?

# 4 Report

Write a report of maximum 4 pages (including text and figures) to discuss the exercises in sections 1, 2 and 3.